In [ ]:
import pywavefront

# データ読み込み

In [ ]:
data = 'data/sample_data.obj'

In [ ]:
scene = pywavefront.Wavefront(data)

In [ ]:
len(scene.vertices)

In [ ]:
import pandas as pd

In [ ]:
coordinates = ['x','y','z']
colors = ['r','g','b']
df = pd.DataFrame(scene.vertices,columns=coordinates + colors)
df.describe()

# `(x,y,z)`座標を0起点に変換

In [ ]:
df[coordinates] = df[coordinates] - df[coordinates].apply(min)
'''
for coor in coordinates:
    df[coor] = df[coor] - min(df[coor])
'''
df[coordinates].describe()

# `(x,y,z)`座標の拡大倍率を決定する

In [ ]:
%%time
max_series = df[coordinates].apply(max)
max_val = max_series.max()
max_idx = max_series.idxmax()
'''
for coor in coordinates:
    if max_val < max(df[coor]):
        max_val = max(df[coor])
        max_idx = coor
'''
print('col:',max_idx)
print('max_val:',max_val)

In [ ]:
HEIGHT_MAX = 256
WIDTH_MAX = 256
if max_idx in ['x','z']:
    zoom_ratio = WIDTH_MAX / max_val
else:
    zoom_ratio = HEIGHT_MAX / max_val
print('zoom_ratio:',zoom_ratio)

In [ ]:
df[coordinates] = df[coordinates] * zoom_ratio
#for coor in coordinates:
#    df[coor] = df[coor] * zoom_ratio
df[coordinates].describe()

# 外接矩形領域の終点

In [ ]:
bounds_e = df[coordinates].apply(max)
print(bounds_e)

# 色情報を復元
値が`1.0`の場合に最大値(`255`)となるように変換  
※これであってるのかは不明

In [ ]:
COLOR_MAX = 255
norm_colors = ['norm_r','norm_g','norm_b']
df[norm_colors] = df[colors] * COLOR_MAX
#for color in colors:
#    df[color] = df[color] * COLOR_MAX
df[colors + norm_colors].describe()

# 座標値を整数に変換（切り捨て）

In [ ]:
import numpy as np
df[coordinates] = df[coordinates].apply(np.floor).astype(int)
df[coordinates].describe()

# ボクセル位置を表す識別子用の列を追加

In [ ]:
def gen_coordinates_id(row):
    return ','.join([str(v) for v in row.values])

In [ ]:
%%time
coor_id = 'coordinate_id'
df[coor_id] = df[coordinates].apply(gen_coordinates_id,axis=1)
df[coor_id]

# 各ボクセルの色の代表値を決める

In [ ]:
%%time
from tqdm import tqdm
typical_colors = {}
for coor_id_val in tqdm(set(df[coor_id])):
    target = df[df[coor_id] == coor_id_val][colors]
    typical_colors[coor_id_val] = target.mean()
#typical_colors

In [ ]:
voxel = np.zeros(list(bounds_e.apply(np.ceil).values.astype(int)) + [3])
voxel.shape

In [ ]:
for coor,color in tqdm(typical_colors.items()):
    x,y,z = [int(v) for v in coor.split(',')]
    voxel[x,y,z] = color[colors].values

In [ ]:
'''
import numpy as np
from math import ceil
typycal_colors = np.zeros((ceil(bounds_e[0]),ceil(bounds_e[1]),ceil(bounds_e[2]),3))
typycal_colors.shape
'''

In [ ]:
'''
from math import floor
for idx in idxs:
    df[idx] = np.floor(df[idx])
'''

In [ ]:
'''
from tqdm import tqdm
for x in tqdm(range(ceil(bounds_e[0]))):
    #for y in tqdm(range(ceil(bounds_e[1]))):
    for y in range(ceil(bounds_e[1])):
        #for z in tqdm(range(ceil(bounds_e[2]))):
        for z in range(ceil(bounds_e[2])):
            target = df[(df['x'] == x) & (df['y'] == y) & (df['z'] == z)]
            if len(target) == 0:
                continue
            typycal_colors[x,y,z] = [target['r'].mean(),target['g'].mean(),target['b'].mean()]
typycal_colors
'''

In [ ]:
'''
for x in range(ceil(bounds_e[0])):
    for y in range(ceil(bounds_e[1])):
        for z in range(ceil(bounds_e[2])):
            target = df[(df['x'] >= x) & (df['x'] < x+1) & (df['y'] >= y) & (df['y'] < y+1) & (df['z'] >= z) & (df['z'] < z+1)]
            typycal_colors[x,y,z] = [target['r'].mean(),target['g'].mean(),target['b'].mean()]
typycal_colors
'''

# 結果表示

In [ ]:
#%matplotlib inline
#%matplotlib notebook
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.image import imread
from mpl_toolkits.mplot3d import Axes3D

In [ ]:
fig = plt.figure(figsize=(16,16))
ax = fig.add_subplot(111, projection='3d')
for coor,color in tqdm(typical_colors.items()):
    x,y,z = [int(v) for v in coor.split(',')]
    ax.plot(x,z,y,marker='s',color=color)
#ax.scatter3D(np.ravel(x), np.ravel(z), np.ravel(y),s=10,marker='.')
ax.set_title("result")
plt.show()